In [2]:
import os

In [3]:
os.chdir("../")
%pwd


'c:\\Users\\ramak\\OneDrive\\Desktop\\P2\\Image-Caption-Generator'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainingConfig:
    train_dir: Path
    pkl_file: Path
    csv_file_path: Path
    saved_trained_model_dir: Path
    epochs: int
    batch_size: int

In [5]:
from src.ImageCaptionGenerator.constants import *
from src.ImageCaptionGenerator.utils.common import read_yaml, create_directory, save_json

True 
 True


In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.main_dir])

    def get_training_config(self) -> TrainingConfig:
        config = self.config.training
        create_directory([config.model_dir])
        # create_directory([config.train_dir])

        training_config = TrainingConfig(
            train_dir = config.train_dir,
            pkl_file = config.pkl_file,
            csv_file_path = config.csv_file_path,
            saved_trained_model_dir = config.saved_trained_model_dir,
            epochs = self.params.EPOCHS,
            batch_size = self.params.BATCH_SIZE,
        )

        return training_config

In [7]:
import os
from src.ImageCaptionGenerator import logger
import urllib.request as request
from kaggle.api.kaggle_api_extended import KaggleApi
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import re
import csv
from glob import glob
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dropout, Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Add
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16
import pickle
import cv2
import tensorflow as tf
from itertools import islice
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

[2023-12-27 21:02:38,889: INFO: utils: NumExpr defaulting to 8 threads.]


In [ ]:
class Training:
    def __init__(self,config = TrainingConfig):
        self.config = config


    def generator(self, image_dict, caption_dict, max_length, vocab_size):
        while 1:
            for i in range(0,len(caption_dict),self.config.batch_size):
                caption = dict(list(caption_dict.items())[0+i: N+i])
                X, y_in, y_out = self.create_sequences(image_dict,caption,max_length, vocab_size)
                yield [X, y_in], y_out


    def create_sequences(image, caption, max_length,vocab_size):
    #n_samples = 0    
        X,y_in,y_out = [],[],[]
        
        for k, v in caption.items():   
            for i in range(1, len(v)):
                X.append(image[k])
        
                in_seq= [v[:i]]
                out_seq = v[i]
        
                in_seq = pad_sequences(in_seq, maxlen=max_length, padding='post', truncating='post')[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        
                y_in.append(in_seq)
                y_out.append(out_seq)
                
        return np.array(X), np.array(y_in), np.array(y_out)


    def training_model(self, vocab_size, max_length):
        pass
